In [1]:
import pandas as pd

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
%matplotlib inline

/Users/apple/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

In [4]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [5]:
label = pd.read_csv('label_data_5.csv')

In [6]:
#label.head()

In [7]:
#label.shape

In [8]:
label2= pd.read_csv('label_data_5.csv', names=['index', 'category'])
label2.drop('index', axis = 1)

,category
0,35A
1,90Z
2,90D
3,90D
4,90D
5,35A
6,35A
7,90D
8,90D
9,90D


In [9]:
#label2.to_csv('label.csv')

In [10]:
label = label2['category']
label.head()

0    35A
1    90Z
2    90D
3    90D
4    90D
Name: category, dtype: object

In [11]:
new_data = pd.read_csv('feature_data_5.csv')

In [12]:
new_data.head()

,Unnamed: 0,Time,x,y,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,...,city_34,city_35,city_36,city_37,city_38,city_39,city_40,city_41,city_42,city_43
0,0,914,-95.073315,38.941743,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1275,-95.055912,38.943095,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,1398,-95.055912,38.943095,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,3,219,-95.055912,38.943095,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,165,-95.055912,38.943095,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
new_data = new_data.drop(new_data.columns[0], axis = 1)
new_data.head()

,Time,x,y,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,...,city_34,city_35,city_36,city_37,city_38,city_39,city_40,city_41,city_42,city_43
0,914,-95.073315,38.941743,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1275,-95.055912,38.943095,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1398,-95.055912,38.943095,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,219,-95.055912,38.943095,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,165,-95.055912,38.943095,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#new_data.to_csv('data.csv')

In [15]:
#new_data.shape

In [16]:
feature_list = new_data.columns

### In the following analysis, we will do the parameter tuning by using Grid Search.  To save time without reducing the performance too much, sometimes I will use Randomized Search.

### We use XGboost classifier first.

In [17]:
from xgboost import XGBClassifier

In [18]:
default_model = XGBClassifier(objective = 'multi: softmax',
                              learning_rate = 0.4,
                              silent = 0,
                              nthread = 4,
                              max_depth = 6,
                              min_child_weight = 1,
                              gamma = 0,
                              reg_alpha = 0.05,
                              subsample = 0.8,
                              colsample_bytree = 0.8,
                              max_delta_step = 1,
                              scale_pos_weight = 1, 
                              seed = 42
                 )

In [19]:
param_dist_1 = {
    'max_depth': [3,4,6],
    'min_child_weight': [1,3,5]
}

In [20]:
param_dist_4 = {
    'reg_alpha': [0.05, 0.1, 0.2, 0.5, 1],
    'reg_lambda': [0.01, 0.03, 0.05, 0.1, 1, 2]
}

In [21]:
random_search_1 = RandomizedSearchCV(default_model, 
                                    param_distributions = param_dist_1,
                                    n_iter = 5,
                                    cv = 3,
                                    scoring = 'f1_weighted',
                                    verbose = 0,
                                    random_state = 42)

### It takes a lot of time to tune param_dist_1 containing max_depth = 8, so I leave comparing max_depth = 8 and the optimal max_depth chosen from param_1_dist_1 at the end.

In [22]:
#random_search_1.fit(new_data, label)

In [23]:
#random_search_1.best_params_

In [24]:
#random_search_1.best_score_

In [25]:
model_2 = XGBClassifier(objective = 'multi: softmax',
                              learning_rate = 0.4,
                              silent = 0,
                              nthread = 4,
                              max_depth = 3,
                              min_child_weight = 3,
                              gamma = 0,
                              reg_alpha = 0.05,
                              subsample = 0.8,
                              colsample_bytree = 0.8,
                              max_delta_step = 1,
                              scale_pos_weight = 1, 
                              seed = 42
                 )

In [26]:
param_dist_2 = {
    'gamma': [0,0.2,0.4,0.7,1]
}

In [27]:
grid_search_2 = GridSearchCV(model_2, 
                            param_grid = param_dist_2,
                            cv = 3,
                            scoring = 'f1_weighted',
                            verbose = 0)

In [28]:
#grid_search_2.fit(new_data, label)

In [29]:
#grid_search_2.best_params_

In [30]:
#grid_search_2.best_score_

In [31]:
param_dist_3 ={
    'subsample': [0.4, 0.5, 0.8, 1],
    'colsample_bytree': [0.4, 0.6, 0.8, 1]
}

In [32]:
#random_search_2 = RandomizedSearchCV(model_2, 
 #                                   param_distributions = param_dist_3,
  #                                  n_iter = 5,
   #                                 cv = 3,
    #                                scoring = 'f1_weighted',
     #                               verbose = 0,
      #                              random_state = 42)

In [33]:
#random_search_2.fit(new_data, label)

In [34]:
#random_search_2.best_params_    {'colsample_bytree': 0.4, 'subsample': 0.4}

In [35]:
#random_search_2.best_score_   0.12001938556220837

In [36]:
model_3 = XGBClassifier(objective = 'multi: softmax',
                              learning_rate = 0.4,
                              silent = 0,
                              nthread = 4,
                              max_depth = 3,
                              min_child_weight = 3,
                              gamma = 0,
                              reg_alpha = 0.05,
                              subsample = 0.4,
                              colsample_bytree = 0.4,
                              max_delta_step = 1,
                              scale_pos_weight = 1, 
                              seed = 42
                 )

In [37]:
param_dist_4 = {
    'reg_alpha': [0.05, 0.1, 0.2, 0.5, 1],
    'reg_lambda': [0.01, 0.03, 0.05, 0.1, 1, 2]
}

In [38]:
#random_search_3 = RandomizedSearchCV(model_3, 
 #                                   param_distributions = param_dist_4,
  #                                  n_iter = 5,
   #                                 cv = 3,
    #                                scoring = 'f1_weighted',
     #                               verbose = 0,
      #                              random_state = 42)

In [39]:
#random_search_3.fit(new_data, label)

In [40]:
#random_search_3.best_params_     {'reg_alpha': 0.2, 'reg_lambda': 2}

In [41]:
#random_search_3.best_score_       0.12600091227177257

In [42]:
model_4 = XGBClassifier(objective = 'multi: softmax',
                              learning_rate = 0.4,
                              silent = 0,
                              nthread = 4,
                              max_depth = 3,
                              min_child_weight = 3,
                              gamma = 0,
                              reg_alpha = 0.2,
                              reg_lambda = 2,
                              subsample = 0.4,
                              colsample_bytree = 0.4,
                              max_delta_step = 1,
                              scale_pos_weight = 1, 
                              seed = 42
                 )

In [43]:
param_dist_5 = {
    'max_depth': [3,8],
    'max_delta_step': [1,2,3]
}

In [44]:
#random_search_4 = RandomizedSearchCV(model_4, 
 #                                   param_distributions = param_dist_5,
  #                                  n_iter = 5,
   #                                 cv = 3,
    #                                scoring = 'f1_weighted',
     #                               verbose = 0,
      #                              random_state = 42)

In [45]:
#random_search_4.fit(new_data, label)

In [46]:
#random_search_4.best_params_     {'max_delta_step': 1, 'max_depth': 3}

In [47]:
#random_search_4.best_score_        0.12600091227177257

In [48]:
param_dist_6 = {
    'max_depth': [3,8],
    'min_child_weight': [1,3],
    'reg_alpha': [0.05, 0.2],
    'reg_lambda': [1,2],
    'subsample': [0.4, 0.8],
    'colsample_bytree': [0.4, 0.8]
}

In [49]:
#random_search_5 = RandomizedSearchCV(model_4, 
 #                                   param_distributions = param_dist_6,
  #                                  n_iter = 5,
   #                                 cv = 3,
    #                                scoring = 'f1_weighted',
     #                               verbose = 0,
      #                              random_state = 42)

In [50]:
#random_search_5.fit(new_data, label)

In [51]:
#random_search_5.best_params_    {'colsample_bytree': 0.4,
# 'max_depth': 3,
 #'min_child_weight': 1,
 #'reg_alpha': 0.05,
 #'reg_lambda': 1,
 #'subsample': 0.4}

In [52]:
#random_search_5.best_score_    0.11706749775814401

In [53]:
param_dist_7 = {
    'max_depth': [3,8,9]
}

In [54]:
model_5 = XGBClassifier(objective = 'multi: softmax',
                              learning_rate = 0.4,
                              silent = 0,
                              nthread = 4,
                              max_depth = 8,
                              min_child_weight = 1,
                              gamma = 0,
                              reg_alpha = 0.05,
                              subsample = 0.8,
                              colsample_bytree = 0.8,
                              max_delta_step = 1,
                              scale_pos_weight = 1, 
                              seed = 42
                 )

In [55]:
grid_search_3 = GridSearchCV(model_4, 
                            param_grid = param_dist_7,
                            cv = 3,
                            scoring = 'f1_weighted',
                            verbose = 0)

In [56]:
grid_search_3.fit(new_data, label)

/Users/apple/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.4,
       gamma=0, learning_rate=0.4, max_delta_step=1, max_depth=3,
       min_child_weight=3, missing=None, n_estimators=100, nthread=4,
       objective='multi: softmax', reg_alpha=0.2, reg_lambda=2,
       scale_pos_weight=1, seed=42, silent=0, subsample=0.4),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 8, 9]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring='f1_weighted',
       verbose=0)

In [57]:
grid_search_3.best_params_

{'max_depth': 3}

In [58]:
grid_search_3.best_score_

0.12600091227177257

In [59]:
grid_search_4 = GridSearchCV(model_5, 
                            param_grid = param_dist_7,
                            cv = 3,
                            scoring = 'f1_weighted',
                            verbose = 0)

In [60]:
grid_search_4.fit(new_data, label)

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.4, max_delta_step=1, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=100, nthread=4,
       objective='multi: softmax', reg_alpha=0.05, reg_lambda=1,
       scale_pos_weight=1, seed=42, silent=0, subsample=0.8),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 8, 9]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring='f1_weighted',
       verbose=0)

In [61]:
grid_search_4.best_params_

{'max_depth': 3}

In [62]:
grid_search_4.best_score_

0.11209022566103652

### The best classifier is model 4.

In [64]:
predictions = grid_search_4.predict(new_data)

In [65]:
u, index = np.unique(predictions, return_index=True)
u

array(['100', '11B', '120', '13A', '13B', '13C', '200', '220', '23', '23H',
       '240', '250', '26A', '26B', '26C', '270', '280', '290', '35A',
       '35B', '370', '40A', '40B', '520', '90A', '90C', '90D', '90F',
       '90G', '90J', '90Z', 'others'], dtype=object)

In [66]:
len(u)

32

In [68]:
f1_score(label, predictions, average = 'macro')

0.14907334190498819

In [70]:
f1_score(label, predictions, average = 'micro')

0.42647013332508432

In [71]:
from sklearn.metrics import accuracy_score

In [72]:
accuracy_score(label, predictions)

0.42647013332508432

In [73]:
f1_score(label, predictions, average = 'weighted')

0.33327257548602635

In [74]:
from sklearn.metrics import confusion_matrix

In [75]:
confusion_matrix(label, predictions)

array([[   1,    0,    0, ...,    0,   16,    0],
       [   1,    0,    0, ...,    0,   21,    0],
       [   0,    0,    3, ...,    0,    7,    0],
       ..., 
       [   0,    0,    0, ...,    2,   32,    0],
       [   0,    0,    0, ...,    0, 9632,    0],
       [   0,    0,    0, ...,    0,    6,    1]])